In [ ]:
!pip install torchsummary
!pip install --upgrade wandb

In [ ]:
import torch
import torchvision

print("PyTorch Version:", torch.__version__)
print("torchvision Version:", torchvision.__version__)
if torch.cuda.is_available():
  print(torch.cuda.current_device())

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# model.py
import numpy as np
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary

class DeconvMobileNet(nn.Module):
    def __init__(self, num_classes, init_weights):
        super(DeconvMobileNet, self).__init__()

        # Use the specified weights argument
        mobilenet = models.mobilenet_v3_small(weights='IMAGENET1K_V1')
        features = list(mobilenet.features.children())
        classifier = list(mobilenet.classifier.children())

        # Extracting layers from MobileNetV3
        self.conv1 = nn.Sequential(features[0])
        # self.pool1 = nn.MaxPool2d(kernel_size=1, stride=1, padding=0, return_indices=True)# stride 1 instead of 2

        self.conv2 = nn.Sequential(features[1], features[2], features[3])
        # self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True)# stride 1 instead of 2

        self.conv3 = nn.Sequential(features[4], features[5], features[6])
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True)# stride 1 instead of 2

        self.conv4 = nn.Sequential(features[7], features[8], features[9])
        # self.pool4 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True) # stride 1 instead of 2

        self.conv5 = nn.Sequential(features[10], features[11], features[12])
        # self.pool5 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True) # stride 1 instead of 2

        # Conv67 layer
        self.conv67 = nn.Sequential(nn.Conv2d(576, 1024, kernel_size=(1, 1)),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(),
                                    nn.Conv2d(1024, 1000, kernel_size=(1, 1)),
                                    nn.BatchNorm2d(1000),
                                    nn.ReLU())

        # Load weights for conv6 and conv7
        w_conv6 = classifier[0].state_dict()
        w_conv7 = classifier[3].state_dict()

        new_input_channels_conv6 = 96
        w_conv6_adjusted = w_conv6['weight'].unsqueeze(2).unsqueeze(3)  # .view(1024, 576, 1, 1)
        self.conv67[0].weight.data.copy_(w_conv6_adjusted)
        self.conv67[0].bias.data.copy_(w_conv6['bias'])
        self.conv67[3].weight.data.copy_(w_conv7['weight'].view(1000, 1024, 1, 1))
        self.conv67[3].bias.data.copy_(w_conv7['bias'])

        # Define deconvolution layers
        self.deconv67 = nn.Sequential(nn.ConvTranspose2d(1000, 1024, kernel_size=1, stride=1, padding=0),  # Mirrors conv67
                                      nn.BatchNorm2d(1024),
                                      nn.ReLU())

        self.unpool5 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose2d(1024, 576, kernel_size=3, stride=1, padding=1),  # Mirrors conv5
            nn.BatchNorm2d(576),
            nn.ReLU(),
            nn.ConvTranspose2d(576, 576, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(576),
            nn.ReLU(),
            nn.ConvTranspose2d(576, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU()
        )

        self.unpool4 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose2d(96, 96, kernel_size=3, stride=1, padding=1),  # Mirrors conv4
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.ConvTranspose2d(96, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.ConvTranspose2d(96, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU()
        )

        self.unpool3 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose2d(40, 40, kernel_size=3, stride=1, padding=1),  # Mirrors conv3
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.ConvTranspose2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.ConvTranspose2d(40, 24, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU()
        )

        self.unpool2 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose2d(24, 24, kernel_size=3, stride=1, padding=1),  # Mirrors conv2
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.ConvTranspose2d(24, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.unpool1 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv1 = nn.Sequential(
            nn.ConvTranspose2d(16, 16, kernel_size=3, stride=1, padding=1),  # Mirrors conv1
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, num_classes, kernel_size=1, stride=1, padding=0)
        )

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        original = x

        x = self.conv1(x)
        # x, p1 = self.pool1(x)

        x = self.conv2(x)
        # x, p2 = self.pool2(x)

        x = self.conv3(x)
        # x, p3 = self.pool3(x)

        x = self.conv4(x)
        # x, p4 = self.pool4(x)

        x = self.conv5(x)
        # x, p5 = self.pool5(x)

        x = self.conv67(x)
        x = self.deconv67(x)

        # x = self.unpool5(x)
        x_interp = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        x = self.deconv5(x_interp)

        # x = self.unpool4(x, p4)
        x_interp = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        x = self.deconv4(x_interp)

        # x = self.unpool3(x, p3)
        x_interp = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        x = self.deconv3(x_interp)

        # x = self.unpool2(x, p2)
        x_interp = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        x = self.deconv2(x_interp)

        # x = self.unpool1(x, p1)
        x_interp = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        x = self.deconv1(x_interp)

        return x

    def _initialize_weights(self):
        targets = [self.conv67, self.deconv67, self.deconv5, self.deconv4, self.deconv3, self.deconv2, self.deconv1]
        for layer in targets:
            for module in layer.children():
                if isinstance(module, nn.BatchNorm2d):
                    nn.init.constant_(module.weight, 1)
                    nn.init.constant_(module.bias, 0)
                elif isinstance(module, nn.ConvTranspose2d):
                    nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
                    if module.bias is not None:
                        nn.init.constant_(module.bias, 0)


if __name__ == '__main__':
    # Instantiate the model
    num_classes = 21 # 20 classes + background
    if torch.cuda.is_available():
        test_model = DeconvMobileNet(num_classes=num_classes, init_weights=True).to('cuda')
    else:
        test_model = DeconvMobileNet(num_classes=num_classes, init_weights=True)

    # Print model summary
    input_shape = (3, 224, 224)  # Adjust the input shape accordingly
    if torch.cuda.is_available():
        summary(test_model, input_size=input_shape, device='cuda')
    else:
        summary(test_model, input_size=input_shape, device='cpu')

    # Generate random input data (batch size = 1, channels = 3, height = 224, width = 224)
    input_shape = (1, 3, 224, 224)
    if torch.cuda.is_available():
        input_data = torch.randn(input_shape).to('cuda')
    else:
        input_data = torch.randn(input_shape)

    # Forward pass
    wanted_output_shape = (1, num_classes, 224, 224)
    output = test_model(input_data)

    # Print the output shape
    assert output.size() == wanted_output_shape, f"Current output Shape is {output.size()} and should be {wanted_output_shape}"

In [ ]:
# model.py
import numpy as np
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary

class DeconvMobileUNet(nn.Module):
    def __init__(self, num_classes, init_weights):
        super(DeconvMobileUNet, self).__init__()

        # Use the specified weights argument
        mobilenet = models.mobilenet_v3_small(weights='IMAGENET1K_V1')
        features = list(mobilenet.features.children())
        classifier = list(mobilenet.classifier.children())

        # Extracting layers from MobileNetV3
        self.conv1 = nn.Sequential(features[0])
        # self.pool1 = nn.MaxPool2d(kernel_size=1, stride=1, padding=0, return_indices=True)# stride 1 instead of 2

        self.conv2 = nn.Sequential(features[1], features[2], features[3])
        # self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True)# stride 1 instead of 2

        self.conv3 = nn.Sequential(features[4], features[5], features[6])
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True)# stride 1 instead of 2

        self.conv4 = nn.Sequential(features[7], features[8], features[9])
        # self.pool4 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True) # stride 1 instead of 2

        self.conv5 = nn.Sequential(features[10], features[11], features[12])
        # self.pool5 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, return_indices=True) # stride 1 instead of 2

        # Conv67 layer
        self.conv67 = nn.Sequential(nn.Conv2d(576, 1024, kernel_size=(7, 7)),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(),
                                    nn.Conv2d(1024, 1000, kernel_size=(1, 1)),
                                    nn.BatchNorm2d(1000),
                                    nn.ReLU())

        # Load weights for conv6 and conv7
        w_conv6 = classifier[0].state_dict()
        w_conv7 = classifier[3].state_dict()

        new_input_channels_conv6 = 96
        w_conv6_adjusted = w_conv6['weight'].unsqueeze(2).unsqueeze(3)  # .view(1024, 576, 1, 1)
        self.conv67[0].weight.data.copy_(w_conv6_adjusted)
        self.conv67[0].bias.data.copy_(w_conv6['bias'])
        self.conv67[3].weight.data.copy_(w_conv7['weight'].view(1000, 1024, 1, 1))
        self.conv67[3].bias.data.copy_(w_conv7['bias'])

        # Define deconvolution layers
        self.deconv67 = nn.Sequential(nn.ConvTranspose2d(1000, 576, kernel_size=7, stride=1, padding=0),  # Mirrors conv67
                                      nn.BatchNorm2d(576),
                                      nn.ReLU())

        self.unpool5 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose2d(576 + 576, 576, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(576),
            nn.ReLU(),
            nn.ConvTranspose2d(576, 576, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(576),
            nn.ReLU(),
            nn.ConvTranspose2d(576, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU()
        )

        self.unpool4 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose2d(96 + 96, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.ConvTranspose2d(96, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.ConvTranspose2d(96, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU()
        )

        self.unpool3 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose2d(40 + 40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.ConvTranspose2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.ConvTranspose2d(40, 24, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU()
        )

        self.unpool2 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose2d(24 + 24, 24, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.ConvTranspose2d(24, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.unpool1 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv1 = nn.Sequential(
            nn.ConvTranspose2d(16 + 16, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, num_classes, kernel_size=1, stride=1, padding=0)
        )

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        original = x

        cx1 = self.conv1(x)
        # print(f"conv 1: {cx1.shape}")
        cx2 = self.conv2(cx1)
        # print(f"conv 2: {cx2.shape}")
        cx3 = self.conv3(cx2)
        # print(f"conv 3: {cx3.shape}")
        cx4 = self.conv4(cx3)
        # print(f"conv 4: {cx4.shape}")
        cx5 = self.conv5(cx4)
        # print(f"conv 5: {cx5.shape}")

        cx67 = self.conv67(cx5)
        # print(f"conv 67: {cx67.shape}")
        dx67 = self.deconv67(cx67)
        # print(f"deconv 67: {dx67.shape}")

        # x_interp_5 = torch.nn.functional.interpolate(dx67, scale_factor=2, mode='nearest')
        # print(f"interp 5: {x_interp_5.shape}")
        x5 = torch.cat([dx67, cx5], dim=1)
        dx5 = self.deconv5(x5)
        # print(f"deconv 5: {dx5.shape}")

        # x_interp_4 = torch.nn.functional.interpolate(dx5, scale_factor=2, mode='nearest')
        # print(f"interp 4: {x_interp_4.shape}")
        x4 = torch.cat([dx5, cx4], dim=1)
        dx4 = self.deconv4(x4)
        # print(f"deconv 4: {dx4.shape}")

        x_interp_3 = torch.nn.functional.interpolate(dx4, scale_factor=2, mode='nearest')
        # print(f"interp 3: {x_interp_3.shape}")
        x3 = torch.cat([x_interp_3, cx3], dim=1)
        dx3 = self.deconv3(x3)
        # print(f"deconv 3: {dx3.shape}")

        x_interp_2 = torch.nn.functional.interpolate(dx3, scale_factor=2, mode='nearest')
        # print(f"interp 2: {x_interp_2.shape}")
        x2 = torch.cat([x_interp_2, cx2], dim=1)
        dx2 = self.deconv2(x2)
        # print(f"deconv 2: {dx2.shape}")

        x_interp_1 = torch.nn.functional.interpolate(dx2, scale_factor=4, mode='nearest')
        # print(f"interp 1: {x_interp_1.shape}")
        x1 = torch.cat([x_interp_1, cx1], dim=1)
        dx1 = self.deconv1(x1)
        # print(f"deconv 1: {dx1.shape}")

        mask = torch.nn.functional.interpolate(dx1, scale_factor=2, mode='nearest')
        # print(f"mask: {mask.shape}")

        return mask

    def _initialize_weights(self):
        targets = [self.conv67, self.deconv67, self.deconv5, self.deconv4, self.deconv3, self.deconv2, self.deconv1]
        for layer in targets:
            for module in layer.children():
                if isinstance(module, nn.BatchNorm2d):
                    nn.init.constant_(module.weight, 1)
                    nn.init.constant_(module.bias, 0)
                elif isinstance(module, nn.ConvTranspose2d):
                    nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
                    if module.bias is not None:
                        nn.init.constant_(module.bias, 0)


if __name__ == '__main__':
    # Instantiate the model
    num_classes = 21 # 20 classes + background
    if torch.cuda.is_available():
        test_model = DeconvMobileUNet(num_classes=num_classes, init_weights=True).to('cuda')
    else:
        test_model = DeconvMobileUNet(num_classes=num_classes, init_weights=True)

    # Print model summary
    input_shape = (3, 224, 224)  # Adjust the input shape accordingly
    if torch.cuda.is_available():
        summary(test_model, input_size=input_shape, device='cuda')
    else:
        summary(test_model, input_size=input_shape, device='cpu')

    # Generate random input data (batch size = 1, channels = 3, height = 224, width = 224)
    input_shape = (3, 3, 224, 224)
    if torch.cuda.is_available():
        input_data = torch.randn(input_shape).to('cuda')
    else:
        input_data = torch.randn(input_shape)

    # Forward pass
    wanted_output_shape = (3, num_classes, 224, 224)
    output = test_model(input_data)

    # Print the output shape
    assert output.size() == wanted_output_shape, f"Current output Shape is {output.size()} and should be {wanted_output_shape}"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


def make_layers():
    vgg16_bn = models.vgg16_bn(pretrained=True)
    features = list(vgg16_bn.features.children())
    classifier = list(vgg16_bn.classifier.children())

    conv1 = nn.Sequential(*features[:6])
    conv2 = nn.Sequential(*features[7:13])
    conv3 = nn.Sequential(*features[14:23])
    conv4 = nn.Sequential(*features[24:33])
    conv5 = nn.Sequential(*features[34:43])

    conv6 = nn.Conv2d(512, 4096, kernel_size=(7, 7))
    conv7 = nn.Conv2d(4096, 4096, kernel_size=(1, 1))

    w_conv6 = classifier[0].state_dict()
    w_conv7 = classifier[3].state_dict()

    conv6.load_state_dict({'weight':w_conv6['weight'].view(4096, 512, 7, 7), 'bias':w_conv6['bias']})
    conv7.load_state_dict({'weight':w_conv7['weight'].view(4096, 4096, 1, 1), 'bias':w_conv7['bias']})

    return [conv1, conv2, conv3, conv4, conv5, conv6, conv7]


class DeconvNet(nn.Module):
    def __init__(self, num_classes, init_weights):
        super(DeconvNet, self).__init__()

        layers = make_layers()

        self.conv1 = layers[0]
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, return_indices=True)

        self.conv2 = layers[1]
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, return_indices=True)

        self.conv3 = layers[2]
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, return_indices=True)

        self.conv4 = layers[3]
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, return_indices=True)

        self.conv5 = layers[4]
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, return_indices=True)

        self.conv67 = nn.Sequential(layers[5], nn.BatchNorm2d(4096), nn.ReLU(),
                                    layers[6], nn.BatchNorm2d(4096), nn.ReLU())

        self.deconv67 = nn.Sequential(nn.ConvTranspose2d(4096, 512, kernel_size=7, stride=1, padding=0), nn.BatchNorm2d(512), nn.ReLU())

        self.unpool5 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU())

        self.unpool4 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU())

        self.unpool3 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.ConvTranspose2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU())

        self.unpool2 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose2d(128, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU())

        self.unpool1 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.deconv1 = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, num_classes, kernel_size=1, stride=1, padding=0))

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        original = x

        x = self.conv1(x)
        x, p1 = self.pool1(x)

        x = self.conv2(x)
        x, p2 = self.pool2(x)

        x = self.conv3(x)
        x, p3 = self.pool3(x)

        x = self.conv4(x)
        x, p4 = self.pool4(x)

        x = self.conv5(x)
        x, p5 = self.pool5(x)


        x = self.conv67(x)
        x = self.deconv67(x)

        x = self.unpool5(x, p5)
        x = self.deconv5(x)

        x = self.unpool4(x, p4)
        x = self.deconv4(x)

        x = self.unpool3(x, p3)
        x = self.deconv3(x)

        x = self.unpool2(x, p2)
        x = self.deconv2(x)

        x = self.unpool1(x, p1)
        x = self.deconv1(x)

        return x

    def _initialize_weights(self):
        targets = [self.conv67, self.deconv67, self.deconv5, self.deconv4, self.deconv3, self.deconv2, self.deconv1]
        for layer in targets:
            for module in layer:
                if isinstance(module, nn.BatchNorm2d):
                    nn.init.constant_(module.weight, 1)
                    nn.init.constant_(module.bias, 0)
                elif isinstance(module, nn.ConvTranspose2d):
                    nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
                    if module.bias is not None:
                        nn.init.constant_(module.bias, 0)


model = DeconvNet(21, init_weights=True)
input_shape = (3, 224, 224)
summary(model, input_size=input_shape, device='cpu')

In [6]:
# Standard Library Imports
from datetime import datetime
import time
from collections.abc import Sequence
import os
import random
import numbers

# Third-party Library Imports
import wandb
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from google.colab import drive

# PyTorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler
from torchvision.datasets import VOCSegmentation
import torchvision.transforms.v2.functional as F

In [7]:
VOC_CLASSES = np.array([
    "background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car",
    "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "potted plant", "sheep", "sofa", "train", "tv/monitor",
])

VOC_COLORMAP = np.array([
    [0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128], [128, 0, 128],
    [0, 128, 128], [128, 128, 128], [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
    [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128], [0, 64, 0], [128, 64, 0],
    [0, 192, 0], [128, 192, 0], [0, 64, 128],
])

class Mask_Aug:
    """
    Class for applying a series of transformations to an image and its corresponding mask.
    """
    def __init__(self, transforms):
        """
        Initializes the Mask_Aug instance with a list of transformations.

        Args:
        - transforms (list): List of transformation functions to be applied.
        """
        self.transforms = transforms

    def __call__(self, image, mask):
        """
        Applies the list of transformations to the input image and mask.

        Args:
        - image (PIL.Image.Image, mode='RGB'): Input image.
        - mask (PIL.PngImagePlugin.PngImageFile, mode='P'): Input mask.

        Returns:
        - tuple: Tuple containing transformed image and mask.
        """
        for t in self.transforms:
            image, mask = t(image, mask)
        return image, mask

class ToTensor:
    """
    Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    Only applied to image not mask.
    """
    def __call__(self, image, mask):
        """
        Converts the input image to a torch tensor.

        Args:
        - image (PIL.Image.Image, mode='RGB'): Input image.
        - mask (PIL.PngImagePlugin.PngImageFile, mode='P'): Input mask.

        Returns:
        - tuple: Tuple containing transformed image and original mask.
        """
        return F.to_tensor(image), mask

class PILToTensor:
    """
    Converts a PIL Image (H x W x C) to a Tensor of shape (C x H x W).
    Only applied to mask not image.
    """
    def __call__(self, image, mask):
        """
        Converts the input mask to a torch tensor.

        Args:
        - image (PIL.Image.Image, mode='RGB'): Input image.
        - mask (PIL.PngImagePlugin.PngImageFile, mode='P'): Input mask.

        Returns:
        - tuple: Tuple containing original image and transformed mask.
        """
        mask = F.pil_to_tensor(mask)
        # Remove border
        # Or possibility to add ignore_index=255 to loss function
        mask[mask == 255] = 0
        return image, mask


class ToPILImage:
    """
    Converts a torch.*Tensor of shape C x H x W or a numpy ndarray of shape H x W x C to a PIL Image while preserving the value range.
    """
    def __init__(self, mode=None):
        """
        Initializes the ToPILImage instance with an optional mode for the PIL Image.

        Args:
        - mode (str, optional): The mode to use for PIL Image conversion.
        """
        self.mode = mode

    def __call__(self, image, mask):
        """
        Converts the input torch tensor and mask to PIL Image.

        Args:
        - image (torch.Tensor): Input image tensor.
        - mask (torch.Tensor): Input mask tensor.

        Returns:
        - tuple: Tuple containing PIL Image representations of the image and colored mask.
        """
        # Converting the mask tensor values into their corresponding RGB color values based on
        # the mapping defined in VOC_COLORMAP.
        colored_mask = VOC_COLORMAP[mask.cpu()].squeeze(0).astype('uint8')
        return F.to_pil_image(image, self.mode), F.to_pil_image(colored_mask, self.mode)

class ToDtype:
    """
    Converts the datatype of the input image to the specified datatype.
    """
    def __init__(self, dtype, scale=False):
        """
        Initializes the ToDtype instance with a target datatype and an optional scale flag.

        Args:
        - dtype (torch.dtype): Target datatype.
        - scale (bool, optional): Flag indicating whether to scale the image. Default is False.
        """
        self.dtype = dtype
        self.scale = scale

    def __call__(self, image, mask):
        """
        Converts the datatype of the input image.

        Args:
        - image (torch.Tensor): Input image tensor.
        - mask (torch.Tensor): Input mask tensor.

        Returns:
        - tuple: Tuple containing transformed image and original mask.
        """
        if not self.scale:
            return image.to(dtype=self.dtype), mask
        image = F.convert_image_dtype(image, self.dtype)
        return image, mask

class Normalize:
    """
    Normalize a tensor image with mean and standard deviation.
    Only applied to image not mask.
    """
    def __init__(self, mean, std):
        """
        Initializes the Normalize instance with mean and standard deviation values.

        Args:
        - mean (list): List of mean values for each channel.
        - std (list): List of standard deviation values for each channel.
        """
        self.mean = mean
        self.std = std

    def __call__(self, image, mask):
        """
        Applies normalization to the input image.

        Args:
        - image (torch.Tensor): Input image tensor.
        - mask (torch.Tensor): Input mask tensor.

        Returns:
        - tuple: Tuple containing normalized image and original mask.
        """
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image, mask

class Resize(torch.nn.Module):
    """
    Resize the input image to the given size.
    """
    def __init__(self, size, image_interpolation=Image.BILINEAR, mask_interpolation=Image.NEAREST, antialias=True):
        """
        Initializes the Resize instance with size and interpolation options.

        Args:
        - size (int or tuple): Target size for resizing.
        - image_interpolation (int, optional): Interpolation method for image resizing. Default is Image.BILINEAR.
        - mask_interpolation (int, optional): Interpolation method for mask resizing. Default is Image.NEAREST.
        - antialias (bool, optional): Flag indicating whether to use antialiasing. Default is True.
        """
        super().__init__()
        if not isinstance(size, (int, Sequence)):
            raise TypeError('Size should be int or sequence. Got {}'.format(type(size)))
        if isinstance(size, Sequence) and len(size) not in (1, 2):
            raise ValueError('If size is a sequence, it should have 1 or 2 values')
        self.size = size
        self.image_interpolation = image_interpolation
        self.mask_interpolation = mask_interpolation
        self.antialias = antialias

    def forward(self, image, mask):
        """
        Resizes the input image and mask.

        Args:
        - image (torch.Tensor): Input image tensor.
        - mask (torch.Tensor): Input mask tensor.

        Returns:
        - tuple: Tuple containing resized image and mask.
        """
        return F.resize(image, self.size, self.image_interpolation, antialias=self.antialias), F.resize(mask, self.size, self.mask_interpolation, antialias=self.antialias)


class RandomCrop(torch.nn.Module):
    """
    Crop the given image at a random location.
    """
    @staticmethod
    def get_params(image, output_size):
        """
        Returns random parameters for cropping.

        Args:
        - image (torch.Tensor): Input image tensor.
        - output_size (tuple): Desired output size.

        Returns:
        - tuple: Tuple containing parameters (i, j, h, w) for cropping.
        """
        h, w = F.get_size(image)
        th, tw = output_size

        if (h + 1 < th) or (w + 1 < tw):
            raise ValueError('Required crop size {} is larger than input image size {}'.format((th, tw), (h, w)))

        if w == tw and h == th:
            return 0, 0, h, w

        i = torch.randint(0, h - th + 1, size=(1,)).item()
        j = torch.randint(0, w - tw + 1, size=(1,)).item()
        return i, j, th, tw

    def __init__(self, size, padding=None, pad_if_needed=False, fill=0, padding_mode='constant'):
        """
        Initializes the RandomCrop instance with parameters for cropping.

        Args:
        - size (tuple): Desired output size.
        - padding (int or tuple, optional): Padding to be applied before cropping. Default is None.
        - pad_if_needed (bool, optional): Flag indicating whether to pad if needed. Default is False.
        - fill (int, optional): Pixel fill value for padding. Default is 0.
        - padding_mode (str, optional): Padding mode. Default is 'constant'.
        """
        super().__init__()
        self.size = tuple(_setup_size(
            size, error_msg='Please provide only two dimensions (h, w) for size.'))
        self.padding = padding
        self.pad_if_needed = pad_if_needed
        self.fill = fill
        self.padding_mode = padding_mode

    def forward(self, image, mask):
        """
        Applies random cropping to the input image and mask.

        Args:
        - image (torch.Tensor): Input image tensor.
        - mask (torch.Tensor): Input mask tensor.

        Returns:
        - tuple: Tuple containing cropped image and mask.
        """
        if self.padding is not None:
            image = F.pad(image, self.padding, self.fill, self.padding_mode)
            mask = F.pad(mask, self.padding, self.fill, self.padding_mode)

        height, width = F.get_size(image)

        # pad if needed
        if self.pad_if_needed and (width < self.size[1]):
            padding = [self.size[1] - width, 0]
            image = F.pad(image, padding, self.fill, self.padding_mode)
            mask = F.pad(mask, padding, self.fill, self.padding_mode)

        if self.pad_if_needed and (height < self.size[0]):
            padding = [0, self.size[0] - height]
            image = F.pad(image, padding, self.fill, self.padding_mode)
            mask = F.pad(mask, padding, self.fill, self.padding_mode)

        i, j, h, w = self.get_params(image, self.size)

        return F.crop(image, i, j, h, w), F.crop(mask, i, j, h, w)

class RandomHorizontalFlip:
    """
    Randomly flip the input image and mask horizontally.
    """
    def __init__(self, flip_prob=0.5):
        """
        Initializes the RandomHorizontalFlip instance with a flip probability.

        Args:
        - flip_prob (float, optional): Probability of horizontal flip.
        """
        self.flip_prob = flip_prob

    def __call__(self, image, mask):
        """
        Applies random horizontal flip to the input image and mask.

        Args:
        - image (torch.Tensor): Input image tensor.
        - mask (torch.Tensor): Input mask tensor.

        Returns:
        - tuple: Tuple containing flipped image and mask.
        """
        if random.random() < self.flip_prob:
            image = F.hflip(image)
            mask = F.hflip(mask)
        return image, mask

def _setup_size(size, error_msg):
    """
    Helper function to validate and process the size argument.

    Args:
    - size: Size argument to be processed.
    - error_msg (str): Error message to be used in case of validation failure.

    Returns:
    - tuple: Tuple containing processed size.
    """
    if isinstance(size, numbers.Number):
        return int(size), int(size)

    if isinstance(size, Sequence) and len(size) == 1:
        return size[0], size[0]

    if len(size) != 2:
        raise ValueError(error_msg)

    return size

In [14]:
class SemanticSegmentationDataset:
    """
    Custom dataset for semantic segmentation using the VOC dataset.

    Args:
    - root (str): Root directory of the VOC Dataset.
    - year (str): The dataset year, supports years "2007" to "2012".
    - image_set (str): Select the image_set to use.
    - batch_size (int, optional): Batch size for DataLoader. Default is 64.
    - shuffle (bool, optional): If True, shuffles the dataset. Default is True.
    - transforms (callable, optional): Transforms to be applied to the dataset.

    Attributes:
    - batch_size (int): Batch size for DataLoader.
    - transforms (callable): Transforms applied to the dataset.
    - dataset: VOC Segmentation dataset instance.
    - loader: DataLoader instance for the dataset.

    Raises:
    - AssertionError: If input and target shapes do not match the expected shapes.
    - AssertionError: If target values are not within the specified range.
    """
    def __init__(self, root, year, image_set, batch_size=64, shuffle=True, transforms=None):
        self.batch_size = batch_size
        self.transforms = transforms

        self.dataset = VOCSegmentation(root=root,
                                       year=year,
                                       image_set=image_set,
                                       download=True,
                                       transforms=self.transforms)

        self.loader = DataLoader(self.dataset,
                                 batch_size=self.batch_size,
                                 shuffle=shuffle,
                                 pin_memory=True)

        # Asserts
        input_shape = (self.batch_size, 3, 224, 224)
        target_shape = (self.batch_size, 1, 224, 224)
        images, targets = next(iter(self.loader))

        assert images.size() == input_shape, f"Wrong input shape, found {images.size()}, expected {input_shape}"
        assert targets.size() == target_shape, f"Wrong target shape, found {targets.size()}, expected {target_shape}"

        # Verify that all the first element of the first batch verify assert
        mask = targets[0]
        mask_np = mask.numpy()
        unique_values = np.unique(mask_np)

        assert all(0 <= value < num_classes for value in unique_values), "Values are not within the specified range."

    def plot(self, model):
        """
        Plots input image, ground truth, and prediction for a given model.

        Args:
        - model: Semantic segmentation model.
        """
        # Consider only for batch size 1 on test_data
        iter_batch = iter(self.loader)

        for idx in range(3):
            image, target = next(iter_batch)
            if torch.cuda.is_available():
                image, target = image.cuda(gpu_id, non_blocking=True), target.cuda(gpu_id, non_blocking=True)
            prediction = model(image)

            # Display the image, ground truth, and prediction
            plt.figure(figsize=(15, 5))

            input_image, ground_truth = ToPILImage()(image.squeeze(0), target.squeeze(0))
            predicted_mask = prediction.argmax(dim=1).byte()
            c, h, w = predicted_mask.shape
            _, predicted_mask = ToPILImage()(image.squeeze(0), predicted_mask)

            # Input Image
            plt.subplot(1, 3, 1)
            plt.imshow(input_image)
            plt.title('Input Image')
            plt.axis('off')

            # Ground Truth
            plt.subplot(1, 3, 2)
            plt.imshow(ground_truth)
            plt.title('Ground Truth')
            plt.axis('off')

            # Prediction
            plt.subplot(1, 3, 3)
            plt.imshow(predicted_mask)
            plt.title('Prediction')
            plt.axis('off')

            plt.show()

In [17]:
class SemanticSegmentationTrainer:
    """
    Trainer class for semantic segmentation models.

    Args:
    - model: Semantic segmentation model.
    - train_dataset: Training dataset instance.
    - val_dataset: Validation dataset instance.
    - criterion: Loss criterion.
    - optimizer: Model optimizer.
    - model_directory (str): Directory to save the model and optimizer checkpoints.
    - num_epochs (int, optional): Number of training epochs. Default is 10.
    - patience (int, optional): Patience for early stopping. Default is 500.
    - gpu_id (int, optional): GPU device ID. Default is 0.

    Attributes:
    - model: Semantic segmentation model.
    - model_is_best (bool): Flag indicating if the model is the best so far.
    - train_dataset: Training dataset instance.
    - val_dataset: Validation dataset instance.
    - criterion: Loss criterion.
    - optimizer: Model optimizer.
    - model_directory (str): Directory to save the model and optimizer checkpoints.
    - num_epochs (int): Number of training epochs.
    - patience (int): Patience for early stopping.
    - early_stopping_counter (int): Counter for early stopping.
    - best_val_loss (float): Best validation loss.
    - eps (float): Small value to prevent division by zero.
    - gpu (int): GPU device ID.
    """
    def __init__(self, model, train_dataset, val_dataset, criterion, optimizer, model_directory, num_epochs=10, patience=500, gpu_id=0):
        self.model = model
        self.model_is_best = False
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.criterion = criterion
        self.optimizer = optimizer
        self.model_directory = model_directory
        self.num_epochs = num_epochs
        self.patience = patience
        self.early_stopping_counter = 0
        self.best_mIoU_loss = float('inf')
        self.eps = 1e-10  # prevent division by 0
        self.gpu = gpu_id

    def save_model_and_optimizer(self, epoch, train_loss, val_loss, mIoU):
        """
        Save model and optimizer checkpoint and log to WandB.

        Args:
        - epoch (int): Current training epoch.
        - train_loss (float): Training loss.
        - val_loss (float): Validation loss.
        - mIoU (float): Mean Intersection over Union.
        """
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'training_loss': train_loss,
            'val_loss': val_loss,
            'mIoU': mIoU
        }

        model_path = f"{self.model_directory}/checkpoint_{epoch}.pth"
        torch.save(checkpoint, model_path)

        # Create a Model Version in WandB
        art = wandb.Artifact(f"mobileNet-ss-{wandb.run.id}", type="model")
        art.add_file(model_path, "model.pth")
        # Log the Version
        wandb.log_artifact(art, aliases=["latest", "best"])

        print(f'Model and optimizer saved at epoch {epoch} with validation loss {val_loss}, mIoU {mIoU}')

    def mIoU(self, output, y):
        """
        Calculate Mean Intersection over Union (mIoU).

        Args:
        - output: Model output.
        - y: Ground truth.

        Returns:
        - tuple: Tuple containing true positives, false positives, and false negatives.
        """
        # only consider for batch size 1 on val_data
        predict = torch.argmax(output, dim=1)
        if torch.cuda.is_available():
            true_positive = torch.zeros(num_classes).cuda(self.gpu, non_blocking=True)
            false_positive = torch.zeros(num_classes).cuda(self.gpu, non_blocking=True)
            false_negative = torch.zeros(num_classes).cuda(self.gpu, non_blocking=True)
        else:
            true_positive = torch.zeros(num_classes)
            false_positive = torch.zeros(num_classes)
            false_negative = torch.zeros(num_classes)

        for i in range(num_classes):
            positive_i = predict == i
            true_i = y == i
            true_positive[i] += torch.sum(torch.logical_and(positive_i, true_i))
            false_positive[i] += torch.sum(torch.logical_and(positive_i, ~true_i))
            false_negative[i] += torch.sum(torch.logical_and(~positive_i, true_i))

        return true_positive, false_positive, false_negative

    def early_stopping(self, mIoU):
        """
        Check for early stopping based on mIoU loss.

        Args:
        - mIoU (float): Current mIoU loss.

        Returns:
        - bool: True if early stopping criteria met, False otherwise.
        """
        if mIoU < self.best_mIoU_loss:
            self.model_is_best = True
            self.best_mIoU_loss = mIoU
            self.early_stopping_counter = 0
        else:
            self.model_is_best = False
            self.early_stopping_counter += 1

        return self.early_stopping_counter >= self.patience

    def train(self):
        """Main training loop."""
        for epoch in range(self.num_epochs):
            start_training_time = time.time()  # Record start time for the epoch
            self.model.train()
            losses = list()

            for images, targets in self.train_dataset.loader:
                batch, channel, height, width = targets.shape
                # the target tensor should have the shape (batch_size, height, width)
                targets = targets.view(batch, height, width).type(torch.LongTensor)

                # Move the tensors to the GPU
                if torch.cuda.is_available():
                    images, targets = images.cuda(self.gpu, non_blocking=True), targets.cuda(self.gpu, non_blocking=True)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                loss.backward()
                self.optimizer.step()

                losses.append(loss.item())

            avg_training_loss = sum(losses) / len(losses)
            # Compute time per epoch
            elapsed_training_time = time.time() - start_training_time

            # Validation
            self.model.eval()
            losses = list()
            if torch.cuda.is_available():
                true_positives = torch.zeros(num_classes).cuda(self.gpu, non_blocking=True)
                false_positives = torch.zeros(num_classes).cuda(self.gpu, non_blocking=True)
                false_negatives = torch.zeros(num_classes).cuda(self.gpu, non_blocking=True)
            else:
                true_positives = torch.zeros(num_classes)
                false_positives = torch.zeros(num_classes)
                false_negatives = torch.zeros(num_classes)

            with torch.no_grad():
                for val_images, val_targets in self.val_dataset.loader:
                    batch, channel, height, width = val_targets.shape
                    # the target tensor should have the shape (batch_size, height, width)
                    val_targets = val_targets.view(batch, height, width).type(torch.LongTensor)

                    # Move the tensors to the GPU
                    if torch.cuda.is_available():
                        val_images, val_targets = val_images.cuda(self.gpu, non_blocking=True), val_targets.cuda(self.gpu, non_blocking=True)

                    val_outputs = self.model(val_images)
                    val_loss = self.criterion(val_outputs, val_targets)
                    losses.append(loss.item())

                    tp, fp, fn = self.mIoU(val_outputs, val_targets)
                    true_positives += tp
                    false_positives += fp
                    false_negatives += fn

            mIoU = torch.sum(true_positives / (self.eps + true_positives + false_positives + false_negatives)) / num_classes
            avg_val_loss = sum(losses) / len(losses)

            print(f"Epoch: {epoch}, training_time: {elapsed_training_time}, train_loss: {avg_training_loss}, val_loss: {avg_val_loss}, mIoU: {mIoU.item()}")
            wandb.log({"epoch": epoch, "training_time": elapsed_training_time, "train_loss": avg_training_loss, "val_loss": avg_val_loss, "mIoU": mIoU.item()})

            # Early stopping check
            if self.early_stopping(mIoU.item()):
                print(f'Early stopping at epoch {epoch} due to no improvement in mIoU loss.')
                break

            # Save checkpoint
            if self.model_is_best:
                self.save_model_and_optimizer(epoch, avg_training_loss, avg_val_loss, mIoU.item())

In [ ]:
def load_model_and_optimizer(model, optimizer, path="checkpoint_{idx_epoch}.pth"):
    if torch.cuda.is_available():
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    training_loss = checkpoint['training_loss']
    val_loss = checkpoint['val_loss']
    mIoU = checkpoint['mIoU']

    print(f'Model and optimizer loaded from epoch {epoch} with validation loss {val_loss}, mIoU {mIoU}')

    return model, optimizer

class SemanticSegmentationPipeline:
    """
    Semantic segmentation pipeline for training and evaluation.

    Args:
    - num_classes (int): Number of classes in the segmentation task.
    - train_batch_size (int): Batch size for training.
    - val_batch_size (int): Batch size for validation.
    - num_epochs (int): Number of training epochs.
    - root (str): Root directory of the VOC Dataset. Default is './data'.
    - year (str): The dataset year, supports years "2007" to "2012". Default is '2012'.
    - gpu_id (int): GPU device ID. Default is 0.

    Attributes:
    - root (str): Root directory of the VOC Dataset.
    - year (str): The dataset year.
    - num_classes (int): Number of classes in the segmentation task.
    - train_batch_size (int): Batch size for training.
    - val_batch_size (int): Batch size for validation.
    - num_epochs (int): Number of training epochs.
    - gpu (int): GPU device ID.

    Methods:
    - run(): Executes the semantic segmentation pipeline.

    Raises:
    - AssertionError: If the GPU device ID is invalid.
    """
    def __init__(self, num_classes, train_batch_size, val_batch_size, num_epochs, root='./data', year='2012', gpu_id=0):
        self.root = root
        self.year = year
        self.num_classes = num_classes
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.num_epochs = num_epochs

        # Set GPU device
        if torch.cuda.is_available():
            assert 0 <= gpu_id < torch.cuda.device_count(), f"Invalid GPU device ID {gpu_id}."
            self.gpu = gpu_id
            torch.cuda.set_device(self.gpu)

    def run(self):
        """
        Execute the semantic segmentation pipeline.
        Initializes WandB, sets up data transformations, datasets, model, loss, optimizer, and trainer.
        Trains the model and logs results.
        """
        wandb.init(
            project="semantic-segmentation",
            config={
                "num_classes": self.num_classes,
                "num_epochs": self.num_epochs,
                "learning_rate": lr,
                "momentum": momentum,
                "weight_decay": weight_decay,
                "train_batch_size": self.train_batch_size,
                "val_batch_size": self.val_batch_size,
            },
        )

        # Define data transforms
        train_tf = Mask_Aug(transforms=[ToTensor(),
                                        PILToTensor(),
                                        Resize((256, 256)), RandomCrop((224, 224)),
                                        RandomHorizontalFlip(0.5),
                                        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        val_tf = Mask_Aug(transforms=[ToTensor(),
                                      PILToTensor(),
                                      Resize((224, 224)),
                                      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        # Create training and validation datasets
        train_dataset = SemanticSegmentationDataset(root=self.root,
                                                    year=self.year,
                                                    image_set='train',
                                                    batch_size=self.train_batch_size,
                                                    shuffle=True,
                                                    transforms=train_tf)

        val_dataset = SemanticSegmentationDataset(root=self.root,
                                                  year=self.year,
                                                  image_set='val',
                                                  batch_size=self.val_batch_size,
                                                  shuffle=False,
                                                  transforms=val_tf)

        # Create the segmentation model
        if torch.cuda.is_available():
            model = DeconvMobileNet(num_classes=self.num_classes, init_weights=True).cuda(self.gpu)
            # model = DeconvMobileUNet(num_classes=self.num_classes, init_weights=True).cuda(self.gpu)
        else:
            model = DeconvMobileNet(num_classes=self.num_classes, init_weights=True)
            # model = DeconvMobileUNet(num_classes=self.num_classes, init_weights=True)

        # Define loss and optimizer
        if torch.cuda.is_available():
            criterion = nn.CrossEntropyLoss().cuda(self.gpu)
        else:
            criterion = nn.CrossEntropyLoss()

        optimizer = optim.SGD(model.parameters(),
                              lr=lr,
                              momentum=momentum,
                              weight_decay=weight_decay)

        # optimizer = optim.AdamW(model.parameters(),
        #          lr=lr,
        #          weight_decay=weight_decay)

        # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0)

        # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)


        # Load previous best model and optimizer state
        # model, optimizer = load_model_and_optimizer(model,
        #                                           optimizer,
        #                                            path='/content/gdrive/My Drive/semantic_segmentation/checkpoint/2023-12-09 08:27/checkpoint_73.pth')

        # Create trainer and start training
        trainer = SemanticSegmentationTrainer(model,
                                              train_dataset,
                                              val_dataset,
                                              criterion,
                                              optimizer,
                                              model_directory,
                                              self.num_epochs)

        trainer.train()

        # Visualize validation results
        val_dataset.plot(model)

if __name__ == "__main__":
    # Mount Google Drive
    drive.mount('/content/gdrive')

    # Provide the path to Google Drive directory
    drive_path = '/content/gdrive/My Drive/semantic_segmentation'

    # Create the directory to save/load model
    current_time = datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M")
    model_directory = drive_path + '/checkpoint/' + formatted_time
    os.makedirs(model_directory, exist_ok=True)

    gpu_id = 0
    num_classes = 21  # 20 classes + background
    num_epochs = 90
    lr = 0.01
    momentum = 0.9
    weight_decay = 0.0005
    train_batch_size = 12
    val_batch_size = 1 # Must be one

    wandb.login()
    pipeline = SemanticSegmentationPipeline(num_classes, train_batch_size, val_batch_size, num_epochs, gpu_id=gpu_id)
    pipeline.run()
    wandb.finish()

# Evaluation

In [ ]:
drive.mount('/content/gdrive')

def load_model_and_optimizer(model, optimizer, path="checkpoint_{idx_epoch}.pth"):
    if torch.cuda.is_available():
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    training_loss = checkpoint['training_loss']
    val_loss = checkpoint['val_loss']
    mIoU = checkpoint['mIoU']

    print(f'Model and optimizer loaded from epoch {epoch} with validation loss {val_loss}, mIoU {mIoU}')

    return model, optimizer

if torch.cuda.is_available():
    model = DeconvMobileNet(num_classes=21, init_weights=True).cuda(0)
else:
    model = DeconvMobileNet(num_classes=21, init_weights=True)

optimizer = optim.SGD(model.parameters(), lr=0.01)

model, _ = load_model_and_optimizer(model, optimizer, path='/content/gdrive/My Drive/semantic_segmentation/checkpoint/2023-12-09 08:27/checkpoint_73.pth')

val_tf = Mask_Aug(transforms=[ToTensor(),
                                      PILToTensor(),
                                      Resize((224, 224)),
                                      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

path_to_data = '/content/gdrive/My Drive/semantic_segmentation/data'
val_dataset = SemanticSegmentationDataset(root='./data',
                                          year='2012',
                                          image_set='val',
                                          batch_size=1,
                                          shuffle=False,
                                          transforms=val_tf)

#image_path = "/content/gdrive/My Drive/semantic_segmentation/data/VOCdevkit/VOC2012/JPEGImages/2008_004917.jpg"
#original_image = Image.open(image_path)
#input, _ = val_tf(original_image, original_image)

val_dataset.plot(model)